In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [3]:
from google.colab import drive
import json
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
MODEL_FILE = "/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2"
METRIC_FILE = "/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model/training_metrics_v2.json"

In [6]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os
import re
from transformers import TrainerCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score
torch.manual_seed(24)
os.environ["WANDB_DISABLED"] = "true"

#!pip install datasets


def remove_wallets(text):
    return ' '.join(word for word in str(text).split() if len(word) < 40)

def clean_text(text):
    # Convert to string and make a working copy
    text = str(text)

    # Remove Asian characters
    text = re.sub(r'[\u4e00-\u9fff]+', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)

    # Remove mentions, hashtags, stock symbols, and forward slashes with words
    text = re.sub(r'[@][A-Za-z0-9_]+', '', text)
    text = re.sub(r'[#][A-Za-z0-9_]+', '', text)
    text = re.sub(r'[$][A-Za-z0-9_ ]+', '', text)
    text = re.sub(r'[/][A-Za-z0-9_ ]+', '', text)

    # Replace specific patterns
    text = re.sub(r'RT : ', '', text)
    text = re.sub(r'&', 'and', text)
    text = re.sub(r'â€™', '\'', text)
    text = re.sub(r'\'', '\'', text)
    text = re.sub(r'', '', text)
    text = re.sub(r'&;', '\'', text)
    text = re.sub(r'\.X', '', text)
    text = re.sub(r'\.x', '', text)

    # Clean up multiple spaces
    text = re.sub(r'  +', ' ', text)

    # Remove @ symbol and pipe characters
    text = re.sub(r'@', '', text)
    text = re.sub(r' \| ', '', text)
    text = re.sub(r'\|', '', text)

    # Normalize multiple dots
    text = re.sub(r'\.\.+', '...', text)

    # Convert to lowercase and remove quotes
    text = text.lower()
    text = re.sub(r'"', '', text)

    # Remove wallets
    text = remove_wallets(text)

    return text.strip()

def sentiment_map(text):
  if 'Bullish' in text:
    return 0
  elif 'Neutral' in text:
    return 1
  else:
    return 2

In [7]:
import torch
import numpy as np
import random

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Create a generator for the split
generator = torch.Generator().manual_seed(seed)


In [8]:
data = load_dataset("StephanAkkerman/financial-tweets-crypto")
train_dataset_ori = data['train']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

crypto.csv:   0%|          | 0.00/54.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/57935 [00:00<?, ? examples/s]

In [9]:

print(f'No. of data: {len(train_dataset_ori)}')
train_dataset_ori = train_dataset_ori.filter(lambda data: data['sentiment'] is not None)
print(f'No. of data after remove sentiment equals to none: {len(train_dataset_ori)}')
train_dataset_ori = train_dataset_ori.filter(lambda data: data['tweet_type']!='quote tweet')
print(f'No. of data after remove quote tweet: {len(train_dataset_ori)}')
train_dataset_ori = train_dataset_ori.filter(lambda data: len(data['description'].split(' '))>1)
print(f'No. of data after remove short text: {len(train_dataset_ori)}')
train_dataset_ori = train_dataset_ori.to_pandas()
train_dataset_ori['description'] = train_dataset_ori['description'].apply(clean_text)
train_dataset_ori.drop_duplicates(inplace=True, ignore_index=True)
print(f'No. of data after remove duplicates: {len(train_dataset_ori)}') # Make sure the records here remains the same after remove duplicates, else the following train test split might be different
train_dataset_ori['sentiment_label'] = train_dataset_ori['sentiment'].apply(sentiment_map)



No. of data: 57935


Filter:   0%|          | 0/57935 [00:00<?, ? examples/s]

No. of data after remove sentiment equals to none: 48692


Filter:   0%|          | 0/48692 [00:00<?, ? examples/s]

No. of data after remove quote tweet: 46866


Filter:   0%|          | 0/46866 [00:00<?, ? examples/s]

No. of data after remove short text: 45567
No. of data after remove duplicates: 45567


In [ ]:
# train_dataset_ori = train_dataset_ori.iloc[:1000]

In [10]:
num_samples = len(train_dataset_ori)
# Create an array of indices
indices = np.arange(num_samples)

# Shuffle the indices randomly
np.random.seed(42)  # Set a seed for reproducibility
np.random.shuffle(indices)

# Split the indices into train, validation, and test sets
train_size = int(num_samples * 0.8)  # 80% for training
val_size = int(num_samples * 0.1)  # 10% for validation
test_size = num_samples - train_size - val_size  # 10% for testing

# Split the shuffled indices
train_idx = indices[:train_size]
val_idx = indices[train_size:train_size + val_size]
test_idx = indices[train_size + val_size:]

# Print the sizes of each split
print(f"Train size: {len(train_idx)}")
print(f"Validation size: {len(val_idx)}")
print(f"Test size: {len(test_idx)}")

Train size: 36453
Validation size: 4556
Test size: 4558


In [11]:
train_dataset = train_dataset_ori.loc[train_idx]
valid_dataset = train_dataset_ori.loc[val_idx]
test_dataset = train_dataset_ori.loc[test_idx]

In [12]:
# 2. Prepare the data
#sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}  # Adjust based on your actual sentiment values

class TweetDataset(Dataset):
    def __init__(self, texts, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        self.texts = texts

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        # print(self.texts[idx])
        item['text'] = self.texts[idx]
        # item['text'] = torch.tensor(self.texts[idx])
        return item

    def __len__(self):
        return len(self.labels)

model_name = "ElKulako/cryptobert"

# 3. Initialize tokenizer
#tokenizer = BertTokenizer.from_pretrained('ElKulako/cryptobert')
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# 6. Create dataset
encodings = tokenizer(train_dataset['description'].to_list(),
                      truncation=True, padding="max_length", max_length=128)
train_dataset = TweetDataset(train_dataset['description'].to_list(), encodings, train_dataset['sentiment_label'].to_list())
encodings = tokenizer(valid_dataset['description'].to_list(),
                      truncation=True, padding="max_length", max_length=128)
val_dataset = TweetDataset(valid_dataset['description'].to_list(), encodings, valid_dataset['sentiment_label'].to_list())
encodings = tokenizer(test_dataset['description'].to_list(),
                      truncation=True, padding="max_length", max_length=128)
test_dataset = TweetDataset(test_dataset['description'].to_list(), encodings, test_dataset['sentiment_label'].to_list())


tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:

# # 11. Define a custom callback to print validation stats
# class PrintValidationStatsCallback(TrainerCallback):
#     def __init__(self, trainer=None):
#         super().__init__()
#         self.trainer = trainer  # Store the trainer instance

#     def on_log(self, args, state, control, logs=None, **kwargs):
#         # Your custom logic
#         super().on_log(args, state, control, **kwargs)

#     # def on_epoch_end(self, args, state, control, **kwargs):
#     # def on_epoch_end(self, args, state, control, **kwargs):
#     def on_evaluate(self, args, state, control, **kwargs):
#         # super().on_log(args, state, control, **kwargs)
#         # super().on_epoch_end(self, args, state, control, **kwargs)  # Propagate logs

#         epoch = state.epoch
#         # Evaluate on the training dataset (using the same compute_metrics)
#         train_metrics = self.trainer.evaluate(
#             eval_dataset=self.trainer.train_dataset,
#             metric_key_prefix="train"  # Prefixes keys with "train_"
#         )
#         # if metrics.get('train_loss'):

#         # Print training stats
#         print(f"\nEpoch {epoch} Training Stats:")
#         print(f"  Training Loss: {train_metrics['train_loss']:.4f}")
#         print(f"  Training Accuracy: {train_metrics['train_accuracy']:.4f}")
#         print(f"  Training F1 Score: {train_metrics['train_f1']:.4f}")
#         print(f"  Training Precision (class 0): {train_metrics['train_precision0']:.4f}")
#         print(f"  Training Precision (class 1): {train_metrics['train_precision1']:.4f}")
#         print(f"  Training Precision (class 2): {train_metrics['train_precision2']:.4f}")

#         # This method is called after each evaluation
#         # trainer = self.trainer#kwargs.get("trainer")
#         eval_metrics = self.trainer.evaluate(eval_dataset=self.trainer.eval_dataset)
#         # if metrics.get('eval_loss'):
#         print(f"\nEpoch {epoch} Validation Stats:")
#         print(f"  Validation Loss: {eval_metrics['eval_loss']:.4f}")
#         print(f"  Validation Accuracy: {eval_metrics['eval_accuracy']:.4f}")
#         print(f"  Validation F1 Score: {eval_metrics['eval_f1']:.4f}")
#         print(f"  Validation Precision (class 0): {eval_metrics['eval_precision0']}")
#         print(f"  Validation Precision (class 1): {eval_metrics['eval_precision1']}")
#         print(f"  Validation Precision (class 2): {eval_metrics['eval_precision2']}")
#         # print(f"  Validation Precision (per class): {[round(p, 4) for p in metrics['eval_precision']]}")

#         # # # Extract the logs from kwargs
#         # # Inject training loss into log history
#         # state.log_history.append({
#         #     'loss': train_metrics['train_loss'],  # Key for training loss
#         #     'epoch': state.epoch,
#         # })

#         # # Propagate logs (critical for table updates)
#         # super().on_epoch_end(args, state, control, **kwargs)



In [13]:
class PrintValidationStatsCallback(TrainerCallback):
    def __init__(self, trainer):
        super().__init__()
        self.trainer = trainer

    def on_evaluate(self, args, state, control, **kwargs):
        # Get validation metrics from the last evaluation (already computed)
        if state.log_history:
          # if len(state.log_history) >= 2:
          #   train_metrics = state.log_history[-2]
          #   train_loss = train_metrics.get("loss", "N/A")
          eval_metrics = state.log_history[-1]  # Assumes evaluation happens at epoch end
          print(eval_metrics)

          # Compute training metrics via prediction (no evaluation loop)
          train_pred = self.trainer.predict(self.trainer.train_dataset)
          train_metrics = self.trainer.compute_metrics(train_pred)
          print(train_metrics)

          print(f"\nEpoch {state.epoch} Training Stats:")
          # if train_loss !='N/A':
          #   print(f"  Training Loss: {train_metrics.get('loss', 'N/A'):.4f}")
          print(f"  Training Accuracy: {train_metrics['accuracy']:.4f}")
          print(f"  Training F1 Score: {train_metrics['f1']:.4f}")
          print(f"  Training Precision (class 0): {train_metrics['precision0']:.4f}")
          print(f"  Training Precision (class 1): {train_metrics['precision1']:.4f}")
          print(f"  Training Precision (class 2): {train_metrics['precision2']:.4f}")

          print(f"\nEpoch {state.epoch} Validation Stats:")
          # print(f"  Validation Loss: {eval_metrics['eval_loss']:.4f}")
          print(f"  Validation Accuracy: {eval_metrics['eval_accuracy']:.4f}")
          print(f"  Validation F1 Score: {eval_metrics['eval_f1']:.4f}")
          print(f"  Validation Precision (class 0): {eval_metrics['eval_precision0']:.4f}")
          print(f"  Validation Precision (class 1): {eval_metrics['eval_precision1']:.4f}")
          print(f"  Validation Precision (class 2): {eval_metrics['eval_precision2']:.4f}")

In [14]:
from copy import deepcopy
import logging
logging.basicConfig(level=logging.INFO)
# 8. Initialize model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 9. Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    # eval_strategy="steps",
    save_strategy="epoch",
    logging_strategy="epoch",  # Log every steps
    evaluation_strategy="epoch", # Evaluate every steps
    # load_best_model_at_end=True,
    # eval_accumulation_steps=1,
    report_to=None,          # Disable external logging (e.g., WANDB),
    # logging_steps=1,#0.2,
    # eval_steps=1,#0.2,
    # save_steps=1,#0.2,
    log_level='debug',
    load_best_model_at_end=True,     # Load the best model (based on metric) at the end
    metric_for_best_model="eval_loss",      # Use F1-score to determine the best model
    greater_is_better=False,          # Higher F1-score is better
    save_total_limit=1,
    # prediction_loss_only=True
)

# # 10. Define compute metrics function
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = np.argmax(pred.predictions, axis=1)

#     # Calculate accuracy
#     accuracy = accuracy_score(labels, preds)

#     # Calculate F1-score (macro average for multi-class)
#     f1 = f1_score(labels, preds, average='macro')
#     prec = precision_score(labels, preds, average=None)
#     return {
#         'accuracy': accuracy,
#         'f1': f1,
#         # 'precision': list(prec)
#     }

def compute_metrics(pred):
      labels = pred.label_ids
      preds = np.argmax(pred.predictions, axis=1)

      # Calculate accuracy
      accuracy = accuracy_score(labels, preds)

      # Calculate F1-score (macro average for multi-class)
      f1 = f1_score(labels, preds, average='macro')
      prec = precision_score(labels, preds, average=None)
      prec = [float(x) for x in prec]
      return {
          # 'loss':
          'accuracy': accuracy,
          'f1': f1,
          'precision0': prec[0],
          'precision1': prec[1],
          'precision2': prec[2]
      }
# 11. Create Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics
# )
# trainer = CustomTrainer(
#  model=model,
#  args=training_args,
#  train_dataset=train_dataset,
#  eval_dataset=val_dataset,
#  compute_metrics=compute_metrics,
# #  callbacks=[CustomMetricsCallback]  # Add this line
# )
# 12. Create Trainer with the callback
trainer = Trainer(  # Assuming CustomTrainer was a typo; replace with your actual CustomTrainer if needed
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    # callbacks=[PrintValidationStatsCallback(trainer=trainer)]  # Add the custom callback
)
# Create the callback and pass the trainer instance
callback = PrintValidationStatsCallback(trainer=trainer)

# Add the callback to the trainer
trainer.add_callback(callback)

# 12. Train the model
trainer.train()


# 13. Evaluate on test set
test_results = trainer.evaluate(val_dataset)
print("\nTest set evaluation results:")
print(f"Test accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Test loss: {test_results['eval_loss']:.4f}")

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Safetensors PR exists
Currently training with a batch size of: 128
***** Running training *****
  Num examples = 36,453
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1,425
  Number of trainable parameters = 124,647,939
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClas

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision0,Precision1,Precision2
1,0.760000,0.680446,0.691176,0.616266,0.751658,0.568299,0.577225
2,0.597500,0.669638,0.704346,0.640178,0.785361,0.536965,0.622857
3,0.455700,0.761030,0.698200,0.626605,0.760308,0.574803,0.585697
4,0.326500,0.880886,0.706321,0.639854,0.777433,0.562227,0.606469
5,0.249700,0.997540,0.695566,0.632481,0.786023,0.534954,0.578815


...

Saving model checkpoint to ./results/checkpoint-1425
Configuration saved in ./results/checkpoint-1425/config.json


{'accuracy': 0.9422818423723699, 'f1': 0.9310642811049293, 'precision0': 0.9805514296323803, 'precision1': 0.8249726709583384, 'precision2': 0.96345116836429}

Epoch 5.0 Training Stats:
  Training Accuracy: 0.9423
  Training F1 Score: 0.9311
  Training Precision (class 0): 0.9806
  Training Precision (class 1): 0.8250
  Training Precision (class 2): 0.9635

Epoch 5.0 Validation Stats:
  Validation Accuracy: 0.6956
  Validation F1 Score: 0.6325
  Validation Precision (class 0): 0.7860
  Validation Precision (class 1): 0.5350
  Validation Precision (class 2): 0.5788


Model weights saved in ./results/checkpoint-1425/model.safetensors
Deleting older checkpoint [results/checkpoint-1140] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-570 (score: 0.6696377992630005).
Deleting older checkpoint [results/checkpoint-1425] due to args.save_total_limit

***** Running Evaluation *****
  Num examples = 4556
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.



***** Running Prediction *****
  Num examples = 36453
  Batch size = 128
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.6696377992630005, 'eval_accuracy': 0.7043459174714662, 'eval_f1': 0.6401775880318773, 'eval_precision0': 0.7853606789250354, 'eval_precision1': 0.5369649805447471, 'eval_precision2': 0.6228571428571429, 'eval_runtime': 28.1508, 'eval_samples_per_second': 161.842, 'eval_steps_per_second': 1.279, 'epoch': 5.0, 'step': 1425}
{'accuracy': 0.8371601788604505, 'f1': 0.8060876970685454, 'precision0': 0.892292264916359, 'precision1': 0.6991171558100692, 'precision2': 0.828604044357469}

Epoch 5.0 Training Stats:
  Training Accuracy: 0.8372
  Training F1 Score: 0.8061
  Training Precision (class 0): 0.8923
  Training Precision (class 1): 0.6991
  Training Precision (class 2): 0.8286

Epoch 5.0 Validation Stats:
  Validation Accuracy: 0.7043
  Validation F1 Score: 0.6402
  Validation Precision (class 0): 0.7854
  Validation Precision (class 1): 0.5370
  Validation Precision (class 2): 0.6229

Test set evaluation results:
Test accuracy: 0.7043
Test loss: 0.6696


In [15]:

# # 13. Save the model
model.save_pretrained(MODEL_FILE)
tokenizer.save_pretrained(MODEL_FILE)


Configuration saved in /content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/config.json
Model weights saved in /content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/model.safetensors
tokenizer config file saved in /content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/special_tokens_map.json


('/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/tokenizer_config.json',
 '/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/special_tokens_map.json',
 '/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/vocab.json',
 '/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/merges.txt',
 '/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/added_tokens.json',
 '/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model_v2/tokenizer.json')

In [16]:
confirm_results = trainer.evaluate(val_dataset)
print("\nTest set evaluation results:")
print(f"Test accuracy: {confirm_results['eval_accuracy']:.4f}")
print(f"Test loss: {confirm_results['eval_loss']:.4f}")


***** Running Evaluation *****
  Num examples = 4556
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.



***** Running Prediction *****
  Num examples = 36453
  Batch size = 128
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.6696377992630005, 'eval_accuracy': 0.7043459174714662, 'eval_f1': 0.6401775880318773, 'eval_precision0': 0.7853606789250354, 'eval_precision1': 0.5369649805447471, 'eval_precision2': 0.6228571428571429, 'eval_runtime': 26.2926, 'eval_samples_per_second': 173.281, 'eval_steps_per_second': 1.369, 'epoch': 5.0, 'step': 1425}
{'accuracy': 0.8371601788604505, 'f1': 0.8060876970685454, 'precision0': 0.892292264916359, 'precision1': 0.6991171558100692, 'precision2': 0.828604044357469}

Epoch 5.0 Training Stats:
  Training Accuracy: 0.8372
  Training F1 Score: 0.8061
  Training Precision (class 0): 0.8923
  Training Precision (class 1): 0.6991
  Training Precision (class 2): 0.8286

Epoch 5.0 Validation Stats:
  Validation Accuracy: 0.7043
  Validation F1 Score: 0.6402
  Validation Precision (class 0): 0.7854
  Validation Precision (class 1): 0.5370
  Validation Precision (class 2): 0.6229

Test set evaluation results:
Test accuracy: 0.7043
Test loss: 0.6696


In [17]:
confirm_results = trainer.evaluate(test_dataset)
print("\nTest set evaluation results:")
print(f"Test accuracy: {confirm_results['eval_accuracy']:.4f}")
print(f"Test loss: {confirm_results['eval_loss']:.4f}")


***** Running Evaluation *****
  Num examples = 4558
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.



***** Running Prediction *****
  Num examples = 36453
  Batch size = 128
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.6684985160827637, 'eval_accuracy': 0.7020623080298376, 'eval_f1': 0.6431659617576958, 'eval_precision0': 0.7889809146561038, 'eval_precision1': 0.5396669931439765, 'eval_precision2': 0.6026315789473684, 'eval_runtime': 28.6506, 'eval_samples_per_second': 159.089, 'eval_steps_per_second': 1.257, 'epoch': 5.0, 'step': 1425}
{'accuracy': 0.8371601788604505, 'f1': 0.8060876970685454, 'precision0': 0.892292264916359, 'precision1': 0.6991171558100692, 'precision2': 0.828604044357469}

Epoch 5.0 Training Stats:
  Training Accuracy: 0.8372
  Training F1 Score: 0.8061
  Training Precision (class 0): 0.8923
  Training Precision (class 1): 0.6991
  Training Precision (class 2): 0.8286

Epoch 5.0 Validation Stats:
  Validation Accuracy: 0.7021
  Validation F1 Score: 0.6432
  Validation Precision (class 0): 0.7890
  Validation Precision (class 1): 0.5397
  Validation Precision (class 2): 0.6026

Test set evaluation results:
Test accuracy: 0.7021
Test loss: 0.6685


In [ ]:
model.load()

In [ ]:
# Extract predictions (logits), probabilities, and true labels
predictions_output = trainer.predict(val_dataset)
logits = predictions_output.predictions  # Raw logits from the model
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()  # Convert logits to probabilities
preds = np.argmax(logits, axis=1)  # Predicted labels
true_labels = predictions_output.label_ids  # True labels

# 14. Evaluate on test set (already computed metrics are available in predictions_output)
test_results = trainer.evaluate(val_dataset)
print("\nTest set evaluation results:")
print(f"Test accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Test loss: {test_results['eval_loss']:.4f}")


***** Running Prediction *****
  Num examples = 9114
  Batch size = 128



***** Running Evaluation *****
  Num examples = 9114
  Batch size = 128



Test set evaluation results:
Test accuracy: 0.7411
Test loss: 0.8707


In [ ]:
# 15. Identify incorrect predictions
incorrect_indices = np.where(preds != true_labels)[0]  # Indices where predictions are wrong
num_incorrect = len(incorrect_indices)

# def sentiment_map(text):
#   if 'Bullish' in text:
#     return 0
#   elif 'Neutral' in text:
#     return 1
#   else:
#     return 2

print(f"\nNumber of incorrect predictions: {num_incorrect}")
print("\nIncorrect predictions:")
for idx in incorrect_indices[:10]:  # Limit to first 10 for brevity
    print('Text: ', texts[idx])#, labels[idx])
    print(f"Sample {idx}:")
    print(f"True label: {true_labels[idx]}")
    print(f"Predicted label: {preds[idx]}")
    print(f"Probabilities: {probs[idx]}")




Number of incorrect predictions: 2360

Incorrect predictions:
Text:  Crazy that $PRIME is going to >$1B market cap at the next cycle top as the “ $AXS of this cycle” and you still don’t own any anon. $75-100 exit or bust bitch.
Sample 0:
True label: 0
Predicted label: 1
Probabilities: [0.00534369 0.51875556 0.4759007 ]
Text:  $AVAX Stacking on support..
Sample 2:
True label: 1
Predicted label: 0
Probabilities: [9.9729925e-01 2.3489676e-03 3.5175710e-04]
Text:  $INJ - Twitter been super bullish on this one and this is the first real pullback since the bottom. Buying the daily re-touch of the supertrend support. Close below and I'll sell.
Sample 3:
True label: 0
Predicted label: 2
Probabilities: [0.02006538 0.00458542 0.97534925]
Text:  $BONK looks ready to get sent
Sample 4:
True label: 2
Predicted label: 1
Probabilities: [0.00489898 0.7302936  0.26480737]
Text:  $BONK negative funding. Shorts go to horny jail.
Sample 12:
True label: 1
Predicted label: 0
Probabilities: [0.6511122  0.06

In [ ]:
from collections import Counter

In [ ]:
Counter(true_labels)

Counter({0: 5600, 1: 1787, 2: 1727})

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the saved model and tokenizer
model_dir = "./trained_bert_model"
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set to evaluation mode

# Function to perform inference
def predict_text(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get model predictions
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(predictions, dim=1).item()

    return predicted_class, predictions

# Example usage
test_text = "This is a great day!"
predicted_class, probabilities = predict_text(test_text)
print(f"Predicted class: {predicted_class}")  # e.g., 1 for positive, 0 for negative
print(f"Probabilities: {probabilities.cpu().numpy()}")

# Test with multiple sentences
test_texts = ["I love this!", "This is awful."]
for text in test_texts:
    pred_class, probs = predict_text(text)
    print(f"Text: {text} | Predicted class: {pred_class} | Probabilities: {probs.cpu().numpy()}")

OSError: Incorrect path_or_model_id: './trained_bert_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
model_test = AutoModelForSequenceClassification.from_pretrained(MODEL_FILE, num_labels=3)

loading configuration file /content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Bearish",
    "1": "Neutral",
    "2": "Bullish"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Bearish": 0,
    "Bullish": 2,
    "Neutral": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the saved model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_FILE)
model = BertForSequenceClassification.from_pretrained(MODEL_FILE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set to evaluation mode

# Function to perform inference
def predict_text(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get model predictions
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(predictions, dim=1).item()

    return predicted_class, predictions

# Example usage
test_text = "This is a great day!"
predicted_class, probabilities = predict_text(test_text)
print(f"Predicted class: {predicted_class}")  # e.g., 1 for positive, 0 for negative
print(f"Probabilities: {probabilities.cpu().numpy()}")

# Test with multiple sentences
test_texts = ["I love this!", "This is awful."]
for text in test_texts:
    pred_class, probs = predict_text(text)
    print(f"Text: {text} | Predicted class: {pred_class} | Probabilities: {probs.cpu().numpy()}")

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
loading configuration file /content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model/config.json
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "_name_or_path": "ElKulako/cryptobert",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Bearish",
    "1": "Neutral",
    "2": "Bullish"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Bearish": 0,
    "Bu

Predicted class: 1
Probabilities: [[0.328277   0.35711053 0.31461248]]
Text: I love this! | Predicted class: 2 | Probabilities: [[0.33292776 0.32864988 0.33842236]]
Text: This is awful. | Predicted class: 1 | Probabilities: [[0.33313963 0.34067747 0.32618293]]


In [ ]:
MODEL_FILE

'/content/gdrive/MyDrive/CS7641-models/trained_cryptobert_model'